## UBER Pickup - Clustering : Kmeans

Nous allons utiliser Kmeans, qui est un algorithme de clustering d'apprentissage non supervisé qui permet de créer des groupes de données en fonction de leur proximité géographique. 

In [1]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.metrics import silhouette_score
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
import plotly
from plotly.graph_objs import Scattermapbox
import plotly.graph_objs as go
import plotly.io as pio
# pio.renderers.default = "svg"
pio.renderers.default = "iframe_connected"

In [2]:
merge_df = pd.read_csv('./src/merge_df.csv')

df_apr14 = pd.read_csv('./src/uber-raw-data-apr14_modified.csv')

In [3]:
merge_sample = merge_df.sample(500000)

On crée un dataset `test` pour faire des tests de clustering

In [5]:
df_test = df_apr14[(df_apr14['Date'] == '2014-04-01')]

In [7]:
coords = ['Lat','Lon']
X = df_test.loc[:,coords]

On scale les données pour faciliter les calculs

In [8]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [9]:
X

array([[ 0.7672978 ,  0.41505401],
       [-0.38027247, -1.10899284],
       [-0.2473388 , -0.20528667],
       ...,
       [-0.11711806, -0.30676215],
       [-0.70039845,  0.69076098],
       [ 0.23285018, -0.00425034]])

### KMEANS

L'algorithme `KMEANS` est un algorithme de clustering qui est un type d’apprentissage non supervisé

Il partitionne les données par un procédé itératif, cad qu'il va répéter une opération autant de fois que nécessaire avant de trouver le meilleur centroide (c'est la moyenne arithmétique de tous les points de données appartenant à un cluster particulier, c'est egalement un point du dataset que l’on choisit comme le « centre » d’un cluster).

* Il va d'abord initialiser un certain nombres de centroides dans un ensemble de données (défini à l'avance).

* Ensuite, il attribut chaque point de donnée au cluster le plus proche en essayant de maintenir les clusters les plus petits possibles et les autres clusters aussi différents que possible.

C'est un algorithme d'esperance-maximisation avec une matrice de covariance, qui utilise la distance euclidienne (directe entre 2 points)

Voici le deroulement de Kmeans:

* **On construit k clusters** : Chaque point est dans le cluster du centroïde qui lui est le plus proche.
* **On calcule les nouveaux centroïdes** : Pour chacun des clusters qu’on vient de former, on calcule la moyenne. Celle-ci devient le nouveau centroïde (n’est pas necessairement un point du jeu de donnée).
* **On recommence jusqu’à ce qu’à ce qu’il y ait convergence** : La convergence correspond au fait que les centroïdes ne changent pas après une mise à jour.

Faisons des tests pour voir le comportement de cet algorithme.

On commence par déterminer un nombre de centroides égale à 6, pour qu'il puisse séparer la zone de densité de clients en 6 groupes

In [10]:
km = KMeans(n_clusters=6)
km.fit(X)

KMeans(n_clusters=6)

In [11]:
df_test['km_clusters'] = km.labels_
df_test.groupby('km_clusters')[['km_clusters']].count().rename(columns={'km_clusters' : 'nb_per_cluster'})

,nb_per_cluster
km_clusters,
0,6161
1,6114
2,577
3,157
4,1136
5,401


In [12]:
fig = px.scatter_mapbox(data_frame=df_test, lat='Lat', lon='Lon', color='km_clusters', mapbox_style='carto-positron', zoom=10.2)
fig.show()

Pour savoir quel nombre de clusters sera le nombre idéal à choisir, il va falloir procéder à plusieurs essais. Nous allons d'abord tester sur plusieurs avec une affluence variable.

D'abord, nous pouvons considérer l'affluence qu'il y a en journée, puis considérer l'affluence qu'il y a en semaine. Nous allons créer plusieurs tranches horaires pour tester notre algorithme et utiliser la méthode Silhouette (avec le coude :elbow) pour déterminer les bonnes valeurs à donner ) KMEANS.

Commençons avec différents essais dans la même journée.

In [13]:
# on choisit la date du 1er avril 2014 pour faire nos tests
df_test = df_apr14[(df_apr14['Date'] == '2014-04-01')]

Nous allons créer plusieurs tranches horaires (6 tranches de 4h pour analyser le trafic)

In [14]:
df_test.head()

,Date/Time,Lat,Lon,Base,Date,month,day_name,day,hour
0,2014-04-01 00:11:00,40.7690,-73.9549,B02512,2014-04-01,4,Tuesday,1,0
1,2014-04-01 00:17:00,40.7267,-74.0345,B02512,2014-04-01,4,Tuesday,1,0
2,2014-04-01 00:21:00,40.7316,-73.9873,B02512,2014-04-01,4,Tuesday,1,0
3,2014-04-01 00:28:00,40.7588,-73.9776,B02512,2014-04-01,4,Tuesday,1,0
4,2014-04-01 00:33:00,40.7594,-73.9722,B02512,2014-04-01,4,Tuesday,1,0


In [70]:
def create_clusters(df , nb_clusters=6 , ranges=4):
    scaler = StandardScaler()

    coords = ['Lat','Lon']
    n,s,liste_dfs,liste_Xs = ranges,[],[],[]

    for i,el in enumerate([i for i in range(24)]):
        if i%n==0:
            s.append(el)

    for i,el in enumerate(s):
        globals() [f'df_test_{i+1}'] = df.loc[(df['hour'] >= el) & (df['hour'] < el+4), :]
        globals() [f'X_{i+1}'] = globals() [f'df_test_{i+1}'].loc[:,coords]
        globals() [f'X_{i+1}'] = scaler.fit_transform(globals() [f'X_{i+1}'])
        liste_dfs.append(globals() [f'df_test_{i+1}'])
        liste_Xs.append(globals() [f'X_{i+1}'])    
        
    km = KMeans(n_clusters=nb_clusters) # on determine un nombre fixe de clusters pour avoir un repère
    liste_df_kmeans, liste_df_groupby_clusters = [], []

    i=0
    while i<len(liste_dfs):
        for df_,X_ in zip(liste_dfs, liste_Xs):
            km.fit(X_)
            df_.reset_index(drop=True)
            df_['km_clusters'] = km.labels_
            globals() [f'df_kmeans_{i+1}'] = df_.groupby('km_clusters')[['km_clusters']].count().rename(columns={'km_clusters' : 'nb_per_cluster'})
            liste_df_kmeans.append(df_)
            liste_df_groupby_clusters.append(globals() [f'df_kmeans_{i+1}'])
            i+=1
            
    return liste_df_kmeans, liste_df_groupby_clusters, liste_Xs
        

In [71]:
liste_df_k, liste_groupby_c, Xs_ = create_clusters(df_test , nb_clusters=6 , ranges=4)

nous avons donc :</br>
* df_test_1 = tranche de  0h /  3h
* df_test_2 = tranche de  4h /  7h
* df_test_3 = tranche de  8h / 11h
* df_test_4 = tranche de 12h / 15h
* df_test_5 = tranche de 16h / 19h
* df_test_6 = tranche de 20h / 23h

Nous allons pouvoir observer les différences entre ces tranches horaires

In [94]:
def clusters_by_range_hours(df, liste_df_kmeans, mapbox_style):


    token = 'pk.eyJ1Ijoib3AzbjVlZCIsImEiOiJjbDllYjl6bGswaG9uM3NsOW0zaGJ4ZHVrIn0.Us-gSPz0QgMbKbPqGkDtjg'
    
    v = [True, False, False, False, False, False]

    fig = go.Figure()

    for i,df_ in enumerate(liste_df_kmeans):
        fig.add_trace(
            go.Scattermapbox(
                lat=df_['Lat'],
                lon=df_['Lon'],
                hovertext=df_['km_clusters'],
                hoverinfo='text',
                mode='markers',
                marker=dict(
#                    size=dict_df.loc[i, 'df_normalized'][dicts['criterion'][i]]*65,
                    color=df_['km_clusters'],
                    colorbar=dict(
                        title=dict(
                            side='right',
                            text='clusters',
                            font=dict(
                                color='black',
                                size=15,
                                family='Arial'
                                )
                            ),
                    #bgcolor='LightSkyBlue',
                    x=1.08,
                    y=0.5,
                    len=1.1)
                    ),
                visible=v[i]
                )
        )

    start = pd.to_datetime(df.loc[0,'Date']).strftime("%m/%d/%Y")
    end = pd.to_datetime(df.loc[554930,'Date']).strftime("%m/%d/%Y")

    fig.update_layout(mapbox_style=mapbox_style, 
                      mapbox_accesstoken=token,
                     title=dict(
                        text=f'Clusters de la ville de Chicago pour la période du {start} au {end}',
                        font=dict(
                            color='rgb(47, 138, 196)',
                            size=16,
                            family='Open Sans'
                        )
                    ))


    fig.update_mapboxes(
        bearing=0,
        center=dict(
            lat=40.7,
            lon=-74
        ),
        pitch=0,
        zoom=9)

    fig.update_layout(
        updatemenus = [go.layout.Updatemenu(
            active = 0,
            #bgcolor = "#4BE8E0",
            #bordercolor = "#4B9AC7",
            buttons = [
                    go.layout.updatemenu.Button(
                        label = "Tranche horaire 0h - 3h",
                        method = "update",
                        args = [{"visible" : [True, False, False, False, False, False]}]),
                    go.layout.updatemenu.Button(
                            label = "Tranche horaire 4h - 7h",
                            method = "update",
                            args = [{"visible" : [False, True, False, False, False, False]}]),
                    go.layout.updatemenu.Button(
                            label = "Tranche horaire 8h - 11h",
                            method = "update",
                            args = [{"visible" : [False, False, True, False, False, False]}]),
                    go.layout.updatemenu.Button(
                            label = "Tranche horaire 12h - 15h",
                            method = "update",
                            args = [{"visible" : [False, False, False, True, False, False]}]),
                    go.layout.updatemenu.Button(
                            label = "Tranche horaire 16h - 19h",
                            method = "update",
                            args = [{"visible" : [False, False, False, False, True, False]}]),
                    go.layout.updatemenu.Button(
                            label = "Tranche horaire 20h - 23h",
                            method = "update",
                            args = [{"visible" : [False, False, False, False, False, True]}])
                ]
            )]
        )
  
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

    fig.show()
     

In [95]:
clusters_by_range_hours(df_test, liste_df_k, mapbox_style="carto-positron")

KMEANS est un algorithme de clusterisation qui fonctionne en paramétrant le nombre clusters que l'on souhaite obtenir(contrairement à DBSCAN qui détermine lui même le nombe de clusters)

Pour trouver ce nombre de clusters optimal K, cad des clusters qui représentent une certaine densité, nous avons le choix entre 2 techniques :

* **Elbow** : Vérifie si les points de données dans un cluster sont proches de leur centroide
* **Silhouette** : Vérifie si les clusters sont éloignés les uns des autres

En souhaitant trouver le nombre optimal de clusters, nous pourrions utiliser 2 paramètres : la somme des ecarts au carré et la distance moyenne de chaque point par rapport à son centroide :

* `Somme des écarts au carré par rapport de chaque observation par rapport à son centroïde d'un même cluster`: 
La somme des carrés intra-vluster est une mesure de la variabilité des observations au sein de chaque cluster. En général, un cluster qui a une petite somme de carrés est plus compact qu'un cluster qui a une grande somme de carrés. Les clusters qui ont des valeurs plus élevées présentent une plus grande variabilité des observations au sein du cluster.

* `Moyenne des distances entre les observations et le centroïde de chaque cluster` : La distance moyenne entre les observations et le centroïde du cluster est une mesure de la variabilité des observations au sein de chaque cluster. En général, un cluster qui a une distance moyenne plus petite est plus compact qu'un cluster qui a une distance moyenne plus grande. Les clusters qui ont des valeurs plus élevées présentent une plus grande variabilité des observations au sein du cluster.

Pour revenir sur nos 2 méthodes,  **Elbow** utilise `uniquement les distances intra-cluster` et **Silhouette** utilise une `combinaison de distances inter-cluster et intra-cluster`. la distance intra-cluster qu'utilise **Elbow** est indiqué dans le paramètre de Kmeans qui s'appelle **inertia**. Pour trouver un compromis à ces 2 différentes méthodes, on peut clusteriser à l'aide de l'inertie et évaluer à l'aide de **Silhouette**.

In [21]:
Xs_[1]

array([[ 0.34559532, -0.65201135],
       [-2.14966807,  0.43497132],
       [-0.37543022, -0.23083215],
       ...,
       [ 0.99966849,  0.44560716],
       [ 0.94559157,  0.44135283],
       [ 0.86576375,  0.39668231]])

Nous allons donc recupérer les valeurs de `KMeans().inertia_` qui correspond à la **distance intra-cluster** de la methode **Elbow** et faire le score de **Silhouette** que nous ploterons dans un graph pour observer les résultats (pour cela nous devons avoir le score de **Silhouette** en fonction des clusters)

Nous allons commencer par tranche de 15 itérations et ajuster (cad augmenter cette valeur) si besoin

In [26]:
def create_silhouette_score_by_range_hour(liste_X_scaled, n_iter=15):
    liste_inertias, liste_sils, liste_Ks = [],[],[]

    i,j=2,0
    while j < len(liste_X_scaled):
        K,sil,inertia = [],[],[]
        for i in range(2,n_iter):
            kmeans = KMeans(n_clusters=i, random_state=0) 
    #        print(j, i)
            kmeans.fit(liste_X_scaled[j])
            K.append(i) 
            inertia.append(kmeans.inertia_) 
            sil.append(silhouette_score(liste_X_scaled[j], kmeans.predict(liste_X_scaled[j])))  
        liste_sils.append(sil)
        liste_Ks.append(K)
        liste_inertias.append(inertia)
        j+=1
    return liste_sils, liste_Ks, liste_inertias

In [27]:
liste_sils, liste_Ks, liste_inertias = create_silhouette_score_by_range_hour(Xs_, n_iter=15)

In [35]:
def elbow_graph(liste_Ks, liste_sils):
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=liste_Ks[0], y=liste_sils[0], mode='lines', name='lines'))

    for i in range(1,len(Ks)):
        fig.add_trace(go.Scatter(x=liste_Ks[i], y=liste_sils[i], mode='lines', name='lines', visible = False))

    fig.update_layout(
            title = go.layout.Title(text = "Score de Elbow en fonction de tranches horaires", x = 0.5),
            showlegend = False)

    fig.update_layout(
        updatemenus = [go.layout.Updatemenu(
            active = 0,
            buttons = [
                        go.layout.updatemenu.Button(
                            label = "Tranche horaire 0h - 3h",
                            method = "update",
                            args = [{"visible" : [True, False, False, False, False, False]}]),
                        go.layout.updatemenu.Button(
                                label = "Tranche horaire 4h - 7h",
                                method = "update",
                                args = [{"visible" : [False, True, False, False, False, False]}]),
                        go.layout.updatemenu.Button(
                                label = "Tranche horaire 8h - 11h",
                                method = "update",
                                args = [{"visible" : [False, False, True, False, False, False]}]),
                        go.layout.updatemenu.Button(
                                label = "Tranche horaire 12h - 15h",
                                method = "update",
                                args = [{"visible" : [False, False, False, True, False, False]}]),
                        go.layout.updatemenu.Button(
                                label = "Tranche horaire 16h - 19h",
                                method = "update",
                                args = [{"visible" : [False, False, False, False, True,  False]}]),
                        go.layout.updatemenu.Button(
                                label = "Tranche horaire 20h - 23h",
                                method = "update",
                                args = [{"visible" : [False, False, False, False, False, True]}])
                    ]
        )]
    )
    
    fig.show()

In [36]:
elbow_graph(liste_Ks, liste_sils)

Les zones de "coudes" sont entre 8 et 12 clusters donc nous pouvons prendre, si nous nous basons seulement sur elbow, un nombre de clusters dans cette tranche, cad en moyenne 10 clusters pour bien séparer la densité de clients.

In [37]:
def silhouette_graph(liste_Ks, liste_inertias):
    fig = go.Figure()

    fig.add_trace(go.Bar(x=liste_Ks[0], y=liste_inertias[0]))

    for i in range(1,len(Ks)):
        fig.add_trace(go.Bar(x=liste_Ks[i], y=liste_inertias[i], visible = False))

    fig.update_layout(
            title = go.layout.Title(text = "Score de Silhouette en fonction de tranches horaires", x = 0.5),
            showlegend = False)

    fig.update_layout(
        updatemenus = [go.layout.Updatemenu(
            active = 0,
            buttons = [
                        go.layout.updatemenu.Button(
                            label = "Tranche horaire 0h - 3h",
                            method = "update",
                            args = [{"visible" : [True, False, False, False, False, False]}]),
                        go.layout.updatemenu.Button(
                                label = "Tranche horaire 4h - 7h",
                                method = "update",
                                args = [{"visible" : [False, True, False, False, False, False]}]),
                        go.layout.updatemenu.Button(
                                label = "Tranche horaire 8h - 11h",
                                method = "update",
                                args = [{"visible" : [False, False, True, False, False, False]}]),
                        go.layout.updatemenu.Button(
                                label = "Tranche horaire 12h - 15h",
                                method = "update",
                                args = [{"visible" : [False, False, False, True, False, False]}]),
                        go.layout.updatemenu.Button(
                                label = "Tranche horaire 16h - 19h",
                                method = "update",
                                args = [{"visible" : [False, False, False, False, True,  False]}]),
                        go.layout.updatemenu.Button(
                                label = "Tranche horaire 20h - 23h",
                                method = "update",
                                args = [{"visible" : [False, False, False, False, False, True]}])
                    ]
        )]
    )
    
    fig.show()

In [38]:
silhouette_graph(liste_Ks, liste_inertias)

#### Conclusion

On voit pour toutes les tranches horaires les mêmes représentations, cad qu'à partir d'un certain nombre de cluster(15) on ne voit plus de changement, donc nous pouvons prendre 15 clusters pour bien répartir les zones en fonction d'une certaine densité.

Nous devons, par contre, choisir un minimum de densité de nos clusters pour éviter aux chauffeurs Uber d'aller dans des zones de faibles regroupements de clients. Pour cela, nous allons calculer puis déterminer un nombre mininum de clients par zone pour afficher correctement ces clusters

Comme nous avions considérés les tranches horaires d'une même journée, nous allons maintenant étudier des clusters selon un jour de la semaine.

### 2 - Etude par 'day_name' pour déterminer le nombre optimal de clusters

In [108]:
df_sample2 = merge_df.sample(500000)
df_sample2.reset_index(drop=True, inplace=True)
df_sample2.head()

,Date/Time,Lat,Lon,Base,Date,month,day_name,day,hour
0,2014-08-15 21:53:00,40.7725,-73.9631,B02598,2014-08-15,8,Friday,15,21
1,2014-06-13 18:20:00,40.7554,-73.9682,B02598,2014-06-13,6,Friday,13,18
2,2014-09-15 06:48:00,40.7368,-74.0010,B02617,2014-09-15,9,Monday,15,6
3,2014-08-14 21:08:00,40.7144,-73.9870,B02598,2014-08-14,8,Thursday,14,21
4,2014-05-17 20:25:00,40.8154,-73.9593,B02512,2014-05-17,5,Saturday,17,20


In [109]:
df_groupby_sample1 = df_sample2.groupby('day_name')[['Date/Time']].count()
df_groupby_sample2 = df_sample2.groupby('day_name')[['Lat','Lon']].mean()
df_groupby_sample1.columns = ['count']
df_groupby_sample = pd.concat([df_groupby_sample1,df_groupby_sample2],axis=1)
df_groupby_sample

,count,Lat,Lon
day_name,,,
Friday,81577,40.739455,-73.974404
Monday,59965,40.739560,-73.970407
Saturday,71149,40.735790,-73.973505
Sunday,54171,40.734039,-73.966291
Thursday,83584,40.741314,-73.974811
Tuesday,72953,40.741237,-73.974453
Wednesday,76601,40.741454,-73.975164


On observe une tendance assez similaire en fonction des jours de la semaine sauf pour le Dimanche ou il y a moins de courses, ce qui est normal vu que c'est un jour ou les gens travaillent moins

Nous allons creer un dataset pour le mois d'avril uniquement en supprimant les clusters qui ne sont pas assez dense : nous allons pour cela créer un filtre qui permettra de pouvoir "mieux densifier" ces clusters

In [101]:
df_test2 = df_apr14.copy()

In [89]:
def create_clusters_by_dayname(df, nb_clusters=15):
    scaler = StandardScaler()

    coords = ['Lat','Lon']
    liste_dfs,liste_Xs = [],[]

    s = [day for day in df['day_name'].unique()]
    s[1],s[-2]  = s[-2],s[1]
    s[-2],s[-1] = s[-1],s[-2]

    for i,el in enumerate(s):
        globals() [f'df_test_{i+1}'] = df.loc[(df['day_name'] == el), :]
        globals() [f'X_{i+1}'] = globals() [f'df_test_{i+1}'].loc[:,coords]
        globals() [f'X_{i+1}'] = scaler.fit_transform(globals() [f'X_{i+1}'])
        liste_dfs.append(globals() [f'df_test_{i+1}'])
        liste_Xs.append(globals() [f'X_{i+1}'])    
        
    km = KMeans(n_clusters=nb_clusters) # on determine un nombre fixe de clusters pour avoir un repère
    liste_df_kmeans, liste_df_groupby_clusters = [], []

    i=0
    while i<len(liste_dfs):
        for df_,X_ in zip(liste_dfs, liste_Xs):
            km.fit(X_)
            df_.reset_index(drop=True)
            df_['km_clusters'] = km.labels_
            globals() [f'df_kmeans_{i+1}'] = df_.groupby('km_clusters')[['km_clusters']].count().rename(columns={'km_clusters' : 'nb_per_cluster'})
            liste_df_kmeans.append(df_)
            liste_df_groupby_clusters.append(globals() [f'df_kmeans_{i+1}'])
            i+=1
            
    return liste_df_kmeans, liste_df_groupby_clusters, liste_Xs
        

In [102]:
kmeans_dayname, groupbys_dayname , Xs_dayname = create_clusters_by_dayname(df_test2, nb_clusters=15)

In [104]:
def plot_clusters_by_dayname(df, liste_df_kmeans, mapbox_style):
    
    df = df.reset_index(drop=True)

    token = 'pk.eyJ1Ijoib3AzbjVlZCIsImEiOiJjbDllYjl6bGswaG9uM3NsOW0zaGJ4ZHVrIn0.Us-gSPz0QgMbKbPqGkDtjg'
    
    v = [True, False, False, False, False, False, False]

    fig = go.Figure()

    for i,df_ in enumerate(liste_df_kmeans):
        fig.add_trace(
            go.Scattermapbox(
                lat=df_['Lat'],
                lon=df_['Lon'],
                hovertext=df_['km_clusters'],
                hoverinfo='text',
                mode='markers',
                marker=dict(
#                    size=dict_df.loc[i, 'df_normalized'][dicts['criterion'][i]]*65,
                    color=df_['km_clusters'],
                    colorbar=dict(
                        title=dict(
                            side='right',
                            text='clusters',
                            font=dict(
                                color='black',
                                size=15,
                                family='Arial'
                                )
                            ),
                    #bgcolor='LightSkyBlue',
                    x=1.08,
                    y=0.5,
                    len=1.1)
                    ),
                visible=v[i]
                )
        )

    start = pd.to_datetime(df.loc[0,'Date']).strftime("%m/%d/%Y")
    end = pd.to_datetime(df.loc[len(df)-1,'Date']).strftime("%m/%d/%Y")

    fig.update_layout(mapbox_style=mapbox_style, 
                      mapbox_accesstoken=token,
                     title=dict(
                        text=f'Clusters de la ville de Chicago pour la période du {start} au {end}',
                        font=dict(
                            color='rgb(47, 138, 196)',
                            size=16,
                            family='Open Sans'
                        )
                    ))


    fig.update_mapboxes(
        bearing=0,
        center=dict(
            lat=40.7,
            lon=-74
        ),
        pitch=0,
        zoom=9)

    fig.update_layout(
        updatemenus = [go.layout.Updatemenu(
            active = 0,
            #bgcolor = "#4BE8E0",
            #bordercolor = "#4B9AC7",
            buttons = [
                    go.layout.updatemenu.Button(
                        label = "Lundi",
                        method = "update",
                        args = [{"visible" : [True, False, False, False, False, False, False]}]),
                    go.layout.updatemenu.Button(
                            label = "Mardi",
                            method = "update",
                            args = [{"visible" : [False, True, False, False, False, False, False]}]),
                    go.layout.updatemenu.Button(
                            label = "Mercredi",
                            method = "update",
                            args = [{"visible" : [False, False, True, False, False, False, False]}]),
                    go.layout.updatemenu.Button(
                            label = "Jeudi",
                            method = "update",
                            args = [{"visible" : [False, False, False, True, False, False, False]}]),
                    go.layout.updatemenu.Button(
                            label = "Vendredi",
                            method = "update",
                            args = [{"visible" : [False, False, False, False, True, False, False]}]),
                    go.layout.updatemenu.Button(
                            label = "Samedi",
                            method = "update",
                            args = [{"visible" : [False, False, False, False, False, True, False]}]),
                    go.layout.updatemenu.Button(
                            label = "Dimanche",
                            method = "update",
                            args = [{"visible" : [False, False, False, False, False, False, True]}])
                ]
            )]
        )
  
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

    fig.show()
     

In [105]:
plot_clusters_by_dayname(df_test2, kmeans_dayname, mapbox_style="carto-positron")

Ce test n'est pas concluant, car il est très difficile de voir une différence en fonction du jour de la semaine (en fonction d'une heure comme une heure de pointe par exemple, il y a une réelle différence avec une heure calme, comme la nuit) : Les cartes concernant les clusters par jour de la semaine sont decoupées en "quartier assez nette" ce qui laisse le choix aux chauffeurs Uber d'utiliser cet algorithme pour déterminer dans quelle zone il souhaite récupérer des clients.

Nous ne pouvons donc pas dire quel est le nombre de cluster optimal (15 quarties, 50 quartiers, 100 quartiers...) : l'énoncé nous impose un délai d'attente maximum de 7 minutes, mais nous ne pouvons pas connaitre, sans calcul de distance et donc de vitesse, le nombre de clusters optimal pour qu'un chauffeur Uber puisse récupérer un client en moins de 7 min après avoir déposé un ancien client (quel est l'intéret de découper en quartier de gros clusters si il y a toujours du monde dans cette zone)

De plus, nous ne connaissons pas la direction vers laquelle un client potentiel souhaite se déplacer : en prenant un cas extrème, comme celui qu'un client parte toujours dans la direction opposée à celle ou se trouve les gros clusters, il faudrait dans se cas inclure les tous petits clusters pour avoir une chance de récupérer un nouveau client en moins de 7 minutes, et il faudrait aussi déterminer un % de chance de partir dans la bonne direction, ce qui permettrai d'ajuster le nombre de clusters que l'on souhaite obtenir.

### 3 - Etude par regroupement de courses pour supprimer les clusters avec densité faible

Tout d'abord, nous allons créer un dataset qui prend un sample de 500k courses pour l'aggréger pouvoir determiner des clusters en fonction de moment .

Cela représente en gros 2 semaines de courses prises de manière aléatoire sur une période de 6 mois (2 semaines car 1/10 de 6 mois)

In [107]:
df_sample2 = merge_df.sample(500000)
df_sample2.reset_index(drop=True, inplace=True)
df_sample2.head()

,Date/Time,Lat,Lon,Base,Date,month,day_name,day,hour
0,2014-09-01 11:52:00,40.7732,-73.9498,B02617,2014-09-01,9,Monday,1,11
1,2014-09-23 19:12:00,40.7145,-73.9444,B02764,2014-09-23,9,Tuesday,23,19
2,2014-07-03 23:06:00,40.7299,-73.9919,B02682,2014-07-03,7,Thursday,3,23
3,2014-09-30 12:06:00,40.6907,-74.1781,B02617,2014-09-30,9,Tuesday,30,12
4,2014-05-21 11:07:00,41.0510,-73.6909,B02598,2014-05-21,5,Wednesday,21,11


In [111]:
scaler = StandardScaler()
coords = ['Lat','Lon']
X = df_sample2.loc[:,coords]
X = scaler.fit_transform(X)

N'oublions pas notre objectif qui est de trouver le nombre de clusters optimal :</br>
* Soit nous conservons le nombre de clusters actuel (15) qui représente les gros quartiers de New York
* Soit nous augmentons le nombre de clusters voulus (par exemple 50) en admettant l'hypothèse qu'un chauffeur Uber reste dans une zone assez restreinte, ce qui permettra un meilleur découpage des quartiers

En prenant en compte la 2e hypothèse, nous affirmons qu'il n'y a pas une saturation de chauffeur Uber dans une petite zone : cad qu'il y a assez de place dans un des 50 clusters pour que le chauffeur puisse y rester (car si cela n'etait pas vrai, il devrait changer de zone et donc le choix de prendre plus de clusters ne serait pas pertinent)

Testons donc avec un nombre de clusters égal à 50 et regardons le resultats (nous allons aussi comparer deux algorithmes de calcul de cluster : **`Algorithme espérance-maximisation ('lloyd')`** et **`Algorithme d'inégalité triangulaire('elkan')`**)

Sans rentrer en details dans ce notebook, car cela le rallongerai davantage, je vais faire une explication sommaire:</br>
* **Algorithme espérance-maximisation** : sépare les points de données en fonction d'une différence de distribution gaussienne
* **Algorithme d'inégalité triangulaire**: éviter le calcul des distances entre points de données qui sont redondantes

Si vous voulez en savoir plus, je vous conseille ces liens qui sont très bien détaillé (tous les liens sont en francais):

**Algorithme espérance-maximisation**:</br>
* https://dridk.me/expectation-maximisation.html
* https://fr.wikipedia.org/wiki/Algorithme_esp%C3%A9rance-maximisation

**Algorithme d'inégalité triangulaire**:
* https://fr.wikipedia.org/wiki/In%C3%A9galit%C3%A9_triangulaire
* https://www.lri.fr/~sebag/Examens_2006/Elkan_resume.doc

In [114]:
kmeans_lloyd = KMeans(n_clusters = 50, algorithm="full")
kmeans_lloyd.fit(X)

KMeans(algorithm='full', n_clusters=50)

In [115]:
kmeans_elkan = KMeans(n_clusters = 50, algorithm="elkan")
kmeans_elkan.fit(X)

KMeans(algorithm='elkan', n_clusters=50)

In [116]:
# numero du clusters avec l'algo esperance-maximisation
df_sample2['cluster_lloyd'] = kmeans_lloyd.labels_

# numero des clusters avec l'algo inégalité triangulaire
df_sample2['cluster_elkan'] = kmeans_elkan.labels_

In [123]:
df_sample2

,Date/Time,Lat,Lon,Base,Date,month,day_name,day,hour,cluster_lloyd,cluster_elkan
0,2014-08-15 21:53:00,40.7725,-73.9631,B02598,2014-08-15,8,Friday,15,21,37,17
1,2014-06-13 18:20:00,40.7554,-73.9682,B02598,2014-06-13,6,Friday,13,18,26,31
2,2014-09-15 06:48:00,40.7368,-74.0010,B02617,2014-09-15,9,Monday,15,6,13,16
3,2014-08-14 21:08:00,40.7144,-73.9870,B02598,2014-08-14,8,Thursday,14,21,4,36
4,2014-05-17 20:25:00,40.8154,-73.9593,B02512,2014-05-17,5,Saturday,17,20,8,34
...,...,...,...,...,...,...,...,...,...,...,...
499995,2014-09-23 21:05:00,40.7507,-73.9818,B02764,2014-09-23,9,Tuesday,23,21,30,49
499996,2014-04-26 23:20:00,40.7097,-74.0603,B02598,2014-04-26,4,Saturday,26,23,36,4
499997,2014-08-30 03:09:00,40.7146,-73.9971,B02598,2014-08-30,8,Saturday,30,3,22,6
499998,2014-09-04 10:48:00,40.7520,-73.9772,B02682,2014-09-04,9,Thursday,4,10,26,49


In [132]:
# regroupement des courses par jour
gb = df_sample2.groupby(['Date','month','day_name','day'])[['Date/Time']].count()
gb.columns = ['nb_courses_per_day']
gb

,,,,nb_courses_per_day
Date,month,day_name,day,
2014-04-01,4,Tuesday,1,1553
2014-04-02,4,Wednesday,2,1959
2014-04-03,4,Thursday,3,2159
2014-04-04,4,Friday,4,2977
2014-04-05,4,Saturday,5,2157
...,...,...,...,...
2014-09-26,9,Friday,26,4247
2014-09-27,9,Saturday,27,4290
2014-09-28,9,Sunday,28,3331


In [143]:
mean_by_day = gb['nb_courses_per_day'].mean()
mean_by_week = mean_by_day * 7
mean_by_week = int(mean_by_week)
mean_by_week

19125

Par semaine, en moyenne, il y 19125 courses

In [140]:
# regroupement des courses par heure
gb2 = df_sample2.groupby(['Date','month','day_name','day','hour'])[['Date/Time']].count()
gb2.columns = ['nb_courses_per_hour']
gb2

nb_courses_per_hour
Date       month day_name day hour                     
2014-04-01 4     Tuesday  1   0                      12
                              1                       5
                              2                       6
                              3                       7
                              4                      17
...                                                 ...
2014-09-30 9     Tuesday  30  18                    282
                              19                    248
                              20                    232
                              21                    221
                              22                    146

[4391 rows x 1 columns]

In [142]:
mean_by_hour = gb2['nb_courses_per_hour'].mean()
mean_by_hour = int(mean_by_hour)
mean_by_hour

113

Pour chaque course, donc pour un point sur la carte, on a attribué un numéro de cluster pour pouvoir les regrouper.

On va regrouper les nombres de clusters par date pour voir si nous devons en retirer pour eviter les zones à faible densité de clients

In [124]:
merge_df.shape

(4534327, 9)

In [175]:
clusters_lloyd = df_sample2.groupby('cluster_lloyd').count()['Date/Time'].values
                 
clusters_elkan = df_sample2.groupby('cluster_elkan').count()['Date/Time'].values

Nous allons donc garder que les clusters qui ont un nombre de points de données supérieur à un certain seuil : nous allons devoir définir ce seuil

Nous allons faire un bref calcul : nous souhaitons qu'un client n'attende que 5 min(pour anecdote, le temps d'un taxi jaune a NY est quasi nul car il y en a beaucoup). De plus, nous allons déterminer que le temps d'une course est égal au temps d'attente d'un client (pour simplifier les calculs) donc il faut 12 clients par heure.

Notre df_sample correspond à peu près à 14 jours de données : sachant que nous voulons minimum 12 clients / heure / cluster, il va falloir décomposer ces 14 jours en :

2 semaines = 14 jours = 14 * 24 heures

donc nous arrivons à : 14 * 24 * 12 clients = 4032 points minimum par cluster

In [181]:
nb_max_data_points = 14*24*12

# 2 semaines ==> 24h * 14j ==> 336h / 6 mois ==> 14h / semaine 
clusters_lloyd > nb_max_data_points

array([ True,  True,  True, False,  True,  True,  True, False,  True,
       False,  True, False, False,  True,  True,  True, False, False,
        True,  True, False, False,  True, False, False, False,  True,
        True, False, False,  True,  True, False, False, False,  True,
       False,  True, False, False, False,  True,  True, False, False,
       False, False,  True, False,  True])

In [182]:
df_sample2['lloyd_is_dense'] = df_sample2['cluster_lloyd'].map(lambda row : 1 if clusters_lloyd[row]>nb_max_data_points else 0)
df_sample2['elkan_is_dense'] = df_sample2['cluster_elkan'].map(lambda row : 1 if clusters_elkan[row]>nb_max_data_points else 0)

In [185]:
df_sample2['lloyd_is_dense'].value_counts()

1    474037
0     25963
Name: lloyd_is_dense, dtype: int64

In [186]:
df_sample2['elkan_is_dense'].value_counts()

1    472543
0     27457
Name: elkan_is_dense, dtype: int64

In [196]:
df_sample2.to_csv('./src/df_sample2.csv', index=False)

In [4]:
df_sample2 = pd.read_csv('./src/df_sample2.csv')

In [5]:
fig = px.scatter_mapbox(df_sample2.loc[df_sample2['lloyd_is_dense']==1,:], lat='Lat', lon='Lon', color='cluster_lloyd', zoom=10, mapbox_style='carto-positron')
fig.show()

On observe que les clusters qui sont vers l'extérieur sont plus étendus(moins dense) que les clusters centraux. Nous pourrions donc améliorer l'algorithme en y ajoutant des contraintes (des poids) sur le prix en fonction de l'eloignement du centre.

In [6]:
test = df_sample2.sample(50000)

In [7]:
df_filter_lloyd = test.loc[test['lloyd_is_dense']==1,:]
df_filter_elkan = test.loc[test['elkan_is_dense']==1,:]

dic = {
    'df' : [df_filter_lloyd, test, df_filter_elkan, test], 
    'color' : ['cluster_lloyd', 'cluster_lloyd', 'cluster_elkan', 'cluster_elkan']
    }

In [11]:
def plot_lloyd_elkan_algo_cluster(df, dic):
    mapbox_access_token = 'pk.eyJ1Ijoib3AzbjVlZCIsImEiOiJjbDllYjl6bGswaG9uM3NsOW0zaGJ4ZHVrIn0.Us-gSPz0QgMbKbPqGkDtjg'

    v = [True, False, False, False]


    fig = go.Figure()

    for df, color in zip(dic['df'],dic['color']):

        fig.add_trace(go.Scattermapbox(
                lat = df.loc[:, 'Lat'],
                lon = df.loc[:, 'Lon'],
                mode = 'markers',
                marker = dict(
                    color=df[color],
                    size = 6,
                    opacity = 0.7,
                    colorbar = dict(
                        titleside = "right",
                        outlinecolor = "rgba(68, 68, 68, 0)"
                    ),
                ),
                text=[color],
            )) 

    fig.update_layout(
        hovermode='closest',
        mapbox=dict(
            accesstoken=mapbox_access_token,
            bearing=0,
            center=go.layout.mapbox.Center(
                lat=40.73,
                lon=-73.9
            ),
            pitch=0,
            zoom=9
        )
    )

    fig.update_layout(
            updatemenus = [go.layout.Updatemenu(
                active = 0,
                #bgcolor = "#4BE8E0",
                #bordercolor = "#4B9AC7",
                buttons = [
                        go.layout.updatemenu.Button(
                            label = "Cluster Lloyd Dense",
                            method = "update",
                            args = [{"visible" : [True, False, False, False]}]),
                        go.layout.updatemenu.Button(
                                label = "Cluster Lloyd Normal",
                                method = "update",
                                args = [{"visible" : [False, True, False, False]}]),
                        go.layout.updatemenu.Button(
                                label = "Cluster Elkan Dense",
                                method = "update",
                                args = [{"visible" : [False, False, True, False]}]),
                        go.layout.updatemenu.Button(
                                label = "Cluster Elkan Normal",
                                method = "update",
                                args = [{"visible" : [False, False, False, True]}])
                    ]
                )]
            )

    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

    fig.show()

In [12]:
plot_lloyd_elkan_algo_cluster(test, dic)

On observe bien une différence entre les clusters Dense (qui ont un nombre minimal de client de 12 par heure) et ceux qui sont plus éparpillés.

Il reste cependant des imperfections, cad des zones dans lesquelles il sera difficile de trouver des clients qui, de plus, sont très éloignés des zones à fortes densités.

**CONCLUSION** :
    
L'algorithme KMEANS arrive bien à créer plusieurs clusters d'une taille similaire pour aider les chauffeurs Uber à trouver de petites zones dans lesquelles ils pourront récupérer des clients et faire de petits trajets, ce qui leur evitera de perdre beaucoup d'essence et de temps pour trouver des clients.

La contrepartie de cet algorithme est que si nous avons beaucoup de chauffeurs dans une même zone, et qu'il se trouve dans des zones à faible densité ou des zones à bordures d'autres zones, il va alors être difficile de savoir vers quelle direction aller.

Enfin, la manière dont KMEANS fonctionne pour la clusterisation des clients est qu'il va créer des cercles plutôt que des découpages de secteurs géographiques, ce qui peut poser problème pour la séparation par quartier des clients. Le fait que des zones à faibles densités persistent, après la selection d'un nombre minimum de client pour une heure donnée, nous oblige à changer de stratégie pour ces zones : devons nous faire des prix différents en fonction de la densité de population de certaines zones (ce qui serait injuste pour ces clients de ne pas habiter dans des endroits plus dense) ?

Cela risque donc de délaisser certaines zones et pénaliser les clients qui seraient entre ces zones car ils ont été filtrés par densité.